In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import torchvision as tv

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

from torch.cuda.amp import autocast, GradScaler

import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
import pandas as pd

In [62]:
X_array =  np.load('X_array.npy')
Y_array =  np.load('Y_array.npy')


In [63]:
X_train, X_test, y_train, y_test = train_test_split(X_array, Y_array, test_size=0.2, random_state=42)

In [64]:
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

In [65]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

359
359
90
90


In [66]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        # Добавляем каналы (если они отсутствуют)
        if x.dim() == 3:  # Проверяем, что входной тензор имеет размерность [N, H, W]
            x = x.unsqueeze(1)  # Добавляем размерность каналов
            #x = torch.cat([x, x, x], dim=1)  # Дублируем каналы, чтобы получить RGB изображение
        # Интерполируем изображения до размера 224x224
        self.x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Создаем Dataset и DataLoader
dataset_train = CustomDataset(X_train, y_train)
dataset_test = CustomDataset(X_test, y_test)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=64, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=64, shuffle=False)

In [67]:
for x, y in dataloader_train:
    print(x.size())

torch.Size([64, 1, 224, 224])
torch.Size([64, 1, 224, 224])
torch.Size([64, 1, 224, 224])
torch.Size([64, 1, 224, 224])
torch.Size([64, 1, 224, 224])
torch.Size([39, 1, 224, 224])


In [68]:
model_resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model_resnet.eval()

Using cache found in C:\Users\lera-/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\lera-\.virtualenvs\-SNfpu_4o\lib\site-packages\torchvision\models\_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

c:\Users\lera-\.virtualenvs\-SNfpu_4o\lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [69]:
# Заменяем последний полносвязный слой на новый слой, который будет соответствовать вашему набору данных
num_classes = 10  # количество классов в вашем наборе данных
model_resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_resnet.fc = nn.Linear(in_features=512, out_features=num_classes, bias=True)
model_resnet

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [70]:
# for param in model_resnet.parameters():
#     param.data = param.data.double()


In [71]:
# Определение функции потерь и оптимизатора
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_resnet.parameters(), lr=0.0001, betas=(0.9, 0.999))


In [72]:
def accuracy(pred, label, threshold=0.5):
    pred = torch.sigmoid(pred)
    pred_labels = (pred > threshold).float()  # Threshold the predictions
    correct = (pred_labels == label).sum().item()  # Compare predictions with labels
    total = label.size(0) * label.size(1)  # Total number of labels
    return correct / total

In [73]:
num_epochs = 30  # количество эпох для обучения

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    acc_val = 0.0
    
    for matrix, labels in tqdm(dataloader_train, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
        matrix, labels = matrix.to(device), labels.to(device)


        optimizer.zero_grad()
        
        outputs = model_resnet(matrix)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_item = loss.item()
        running_loss += loss_item
        
        
        acc_current = accuracy(outputs.cpu().float(), labels.cpu().float())
        acc_val += acc_current

      
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader_train):.3f}, Accuracy: {acc_val/len(dataloader_train):.3f}") 

Epoch [1/30], Loss: 0.583, Accuracy: 0.728


Epoch [2/30], Loss: 0.473, Accuracy: 0.795


Epoch [3/30], Loss: 0.443, Accuracy: 0.798


Epoch [4/30], Loss: 0.411, Accuracy: 0.827


Epoch [5/30], Loss: 0.398, Accuracy: 0.830


...

Epoch [26/30], Loss: 0.162, Accuracy: 0.933


Epoch [27/30], Loss: 0.153, Accuracy: 0.936


Epoch [28/30], Loss: 0.139, Accuracy: 0.941


Epoch [29/30], Loss: 0.127, Accuracy: 0.948


Epoch [30/30], Loss: 0.125, Accuracy: 0.946


In [74]:
loss_val = 0.0
acc_val = 0.0
for sample in tqdm(dataloader_test):
    matrix, label = sample[0].to(device), sample[1].to(device)
  

    pred = model_resnet(matrix)
    loss = criterion(pred, label)

    loss_item = loss.item()
    loss_val += loss_item


    acc_current = accuracy(pred.cpu().float(), label.cpu().float())
    acc_val += acc_current


print(f'Loss: {loss_val/len(dataloader_test):.5f}, Accuracy: {acc_val/len(dataloader_test):.3f}')

100%|██████████| 2/2 [00:00<00:00, 18.35it/s]

Loss: 0.95993, Accuracy: 0.764


In [75]:
def aroma_map_comparison(matrix, labels):
    matrix = matrix.unsqueeze(1)
    pred = model_resnet(matrix).to('cpu')
    labels = labels.to('cpu')

    df = pd.DataFrame(dict(
        r=torch.sigmoid(pred).detach().numpy()[0],
        theta=['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus',
       'Fruits ', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']))
    
    df['Label'] = 'Predict'
    
    df1 = pd.DataFrame(dict(
        r=labels.detach().numpy(),
        theta=['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus',
       'Fruits ', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']))
    
    df1['Label'] = 'Experiment'
    
    fig = px.line_polar(pd.concat([df, df1]), color="Label",  r='r', theta='theta', line_close=True, line_shape='linear', color_discrete_sequence=['#008080', '#FFC0CB'],
                    template="plotly_dark")
    

    
    fig.update_traces(fill='toself')
    fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])))
    fig.show()

In [76]:
for sample in dataset_test:
    matrix, label = sample[0].to(device), sample[1].to(device)
    aroma_map_comparison(matrix, label)

: 